# Post Tournament Evaluation and Metrics

In this notebook, we'll take a look at how our bracket did compared to the actual tournament results.

In [1]:
# Import packages
import sys
sys.path.append('../')

import datetime
import pandas as pd
import collegebasketball as cbb
cbb.__version__

'2023'

### Load in Scores from Last Season
Now that the season is complete, we can retrieve all of the postseason scores for both evaluating our bracket this year and as more training data for next year.

In [2]:
# Dates to search for games
year = 2023
start = datetime.date(year, 3, 13)
end = datetime.date(year, 4, 10)

# Set up the path for this years scores
path = '../Data/Scores/'
path_regular = path + str(year) + '_regular_season.csv'
path_full = path + str(year) + '_season.csv'

In [3]:
# Get tournament games from college basketball reference
tournament = cbb.load_scores_dataframe(start, end, csv_file_path=None)
tournament.head(3)

,Home,Away,Home_Score,Away_Score,Tournament
0,Texas A&M-Corpus Christi,Alabama,75.0,96.0,"NCAA Tourn., First Round"
1,Northern Kentucky,Houston,52.0,63.0,"NCAA Tourn., First Round"
2,Howard,Kansas,68.0,96.0,"NCAA Tourn., First Round"


In [4]:
# Load regular season and combine to form file with whole season
regular = pd.read_csv(path_regular)
full_season = pd.concat([regular, tournament], ignore_index=True)
full_season.to_csv(path_full index=False)

In [5]:
data = pd.read_csv(path_full)
data.head()

,Home,Away,Home_Score,Away_Score,Tournament
0,UNC Wilmington,UNC,56.0,69.0,NaN
1,North Florida,Gonzaga,63.0,104.0,NaN
2,Northern Colorado,Houston,36.0,83.0,NaN
3,Howard,Kentucky,63.0,95.0,NaN
4,Mississippi Valley State,Baylor,53.0,117.0,NaN


### Load in Predictions and Kenpom Data for This Season
In addition to the actual tournament game scores, we'll need our predictions and the pre-tournament Kenpom data to evaluate our bracket. The scores are obviously needed to verify when we were correct, but the Kenpom data is also necessary to determine which team was favored in each game since our model determines favorites using the Kenpom efficiency metric rather than tournament seeding.

In [6]:
scores = cbb.filter_tournament(data)
predictions = pd.read_csv(f'../Data/predictions/predictions_{year}.csv')
kenpom = pd.read_csv(f'../Data/Kenpom/{year}_kenpom.csv')
kenpom = cbb.update_kenpom(kenpom)
kenpom.head(3)

,Rank,Team,Seed,Conf,Wins,Losses,AdjEM,AdjO,AdjO Rank,AdjD,...,Luck,Luck Rank,OppAdjEM,OppAdjEM Rank,OppO,OppO Rank,OppD,OppD Rank,NCSOS AdjEM,NCSOS AdjEM Rank
0,1,Houston,1.0,Amer,31,3,28.82,118.4,11,89.6,...,0.025,122,4.21,92,107.3,101,103.1,86,0.63,152
1,2,UCLA,2.0,P12,29,5,27.63,114.9,25,87.2,...,-0.001,196,9.00,33,109.7,32,100.7,44,0.72,149
2,3,Alabama,1.0,SEC,29,5,27.54,116.1,19,88.6,...,0.074,37,11.66,9,110.9,7,99.2,12,10.56,5


### Calculate Metrics

Now that we have all the necessary data, we can use a function to get all the metrics.

In [7]:
# Run evaluation function on our data
cbb.post_tournament_eval(predictions, scores, kenpom)

,Games,Games Contained Predicted Winner,Correct Predictions,Total Upsets,Upsets Predicted,Games Containing Actual Upset Winner,Games Containing Predicted Upset Winner,Correct Upsets,Correct Predicted Upsets,Total Accuracy,Upset Precision,Upset Recall,Adj Accuracy,Adj Upset Precision,Adj Upset Recall
0,32,32,21,9,8,9,8,3,3,0.65625,0.375000,0.333333,0.656250,0.375000,0.333333
1,16,13,7,4,6,1,4,1,1,0.43750,0.166667,0.250000,0.538462,0.250000,1.000000
2,8,3,2,4,4,2,1,1,1,0.25000,0.250000,0.250000,0.666667,1.000000,0.500000
3,4,1,0,3,1,0,0,0,0,0.00000,0.000000,0.000000,0.000000,NaN,NaN
4,2,0,0,0,1,0,0,0,0,0.00000,0.000000,NaN,NaN,NaN,NaN
5,1,0,0,0,1,0,0,0,0,0.00000,0.000000,NaN,NaN,NaN,NaN
0,63,49,30,20,21,12,13,5,5,0.47619,0.238095,0.250000,0.612245,0.384615,0.416667


Some notes on the various metrics above:
* **Games Contained Predicted Winner**: The number of games my bracket's predicted winner actually played in. In later rounds, my predicted winner may have already lost in a previous round so my predicted winner may not have even played in the game.
* **Total Upsets:** The number of actual upsets where an upset is defined as a team with a lower Kenpom efficiency score winning the game. Note this is referring the actual tournament results.
* **Upsets Predicted:** The number of games I predicted an upset based on the two teams I predicted to be playing in my bracket.
* **Games Containing Actual Upset Winner:** The number of games that were actual upsets where my bracket had the winning team predicted to be in the game.
* **Games Containing Predicted Upset Winner:** The number of games I predicted an upset where my predicted upset winner actually played. 
* **Correct Upsets:** The number of games where the actual winner was an underdog that I correctly predicted would win.
* **Correct Predicted Upsets:** The number of games where I predicted an upset and that team actually won the game.
* **Total Accuracy:** The fraction of all games where I correctly predicted the winner.
* **Upset Precision:** The fraction of all upsets I predicted that were correct.
* **Upset Recall:** The fraction of all actual upsets I predicted correctly.
* **Adj Accuracy:** The fraction of all games containing my predicted winner that I predicted correctly.
* **Adj Upset Precision:** The fraction of all games containing my predicted upset winner that I predicted correctly.
* **Adj Upset Recall:** The fraction of all actual upsets containing my predicted winner that I predicted correctly. 

The purpose of these "adjusted" stats are to adjust for the fact that in later rounds, it might not have even been possible to make a correct prediction based on previous errors. While I can see how you would want to know the full accuracy numbers for every game, I think these "adjusted" metrics provide a more accurate measure of performance on a game by game basis so that is why I've calculated them as well.

### How did the Bracket Perform this Year?

Unfortunately, it was somewhat of an off year for this project, though that can partially be explained by the unusual number and unexpectedness of the upsets this year. The overall accuracy, precision and recall were down compared to most previous year. However, the adjusted metrics, which account for games where my selected winner wasn't even present aren't too bad, which I think speaks to how this tournament had some shocking early upsets that shaped the rest of the tournament.

Unfortunately, I haven't found much time to make many improvements for next year, but I'm looking forward to seeing how the bracket performs next tournament.